## Imports

In [73]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [74]:
shapefile_path = "../Data/OpenTransportMap/roadlinks_ES511.shp"
gdf = gpd.read_file(shapefile_path)

gdf

,inspireid,sourceid,beginlifes,endlifespa,validfrom,validto,fictitious,direction,fromroadno,toroadnode,...,speedlimit,capacity,maximuheig,maximumtot,maximumwid,vehicletyp,z_order,length_m,trafficvol,geometry
0,54769333,198794320,2015-10-21,NaT,2015-10-21,NaT,0,bothDirections,33620027,39891066,...,0,2000,0.0,0.0,0.0,None,0,0.0,0.000,"LINESTRING (1.99488 41.66973, 1.99502 41.66971)"
1,54800801,198935562,2015-10-21,NaT,2015-10-21,NaT,0,bothDirections,36092175,39916521,...,50,4000,0.0,0.0,0.0,None,0,0.0,0.000,"LINESTRING (2.23387 41.94304, 2.23386 41.94206)"
2,54802891,198944072,2015-10-21,NaT,2015-10-21,NaT,0,inDirection,39918239,39918240,...,0,3000,0.0,0.0,0.0,None,0,0.0,0.000,"LINESTRING (1.68342 41.33341, 1.68351 41.33332..."
3,54808949,198965364,2015-10-21,NaT,2015-10-21,NaT,0,bothDirections,39923308,39923309,...,0,2000,0.0,0.0,0.0,None,0,0.0,0.000,"LINESTRING (2.01923 41.57089, 2.01926 41.57088)"
4,54808951,198965365,2015-10-21,NaT,2015-10-21,NaT,0,bothDirections,39923311,39923312,...,0,2000,0.0,0.0,0.0,None,0,0.0,0.000,"LINESTRING (2.01857 41.57034, 2.01852 41.57033)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336636,54702654,198501303,2015-10-21,NaT,2015-10-21,NaT,0,bothDirections,39838439,39838440,...,0,6000,0.0,0.0,0.0,None,0,0.0,148.886,"LINESTRING (2.28519 41.683, 2.28473 41.68289)"
336637,40882006,142707200,2015-10-20,NaT,2015-10-20,NaT,0,bothDirections,36532410,23640923,...,0,6000,0.0,0.0,0.0,None,0,0.0,465.900,"LINESTRING (2.31706 41.57213, 2.31722 41.57238)"
336638,54707014,198515267,2015-10-21,NaT,2015-10-21,NaT,0,bothDirections,39738199,39841942,...,0,1500,0.0,0.0,0.0,None,0,0.0,0.000,"LINESTRING (2.01332 41.5798, 2.01332 41.57984,..."
336639,54719488,198595860,2015-10-21,NaT,2015-10-21,NaT,0,bothDirections,40033461,39852110,...,0,1500,0.0,0.0,0.0,None,0,0.0,0.000,"LINESTRING (1.75909 41.24577, 1.75928 41.24584..."


In [75]:
gdf.columns

Index(['inspireid', 'sourceid', 'beginlifes', 'endlifespa', 'validfrom',
       'validto', 'fictitious', 'direction', 'fromroadno', 'toroadnode',
       'roadname', 'nationalro', 'functional', 'formofway', 'roadsurfac',
       'speedlimit', 'capacity', 'maximuheig', 'maximumtot', 'maximumwid',
       'vehicletyp', 'z_order', 'length_m', 'trafficvol', 'geometry'],
      dtype='object')

In [76]:
print(gdf.geometry)


0           LINESTRING (1.99488 41.66973, 1.99502 41.66971)
1           LINESTRING (2.23387 41.94304, 2.23386 41.94206)
2         LINESTRING (1.68342 41.33341, 1.68351 41.33332...
3           LINESTRING (2.01923 41.57089, 2.01926 41.57088)
4           LINESTRING (2.01857 41.57034, 2.01852 41.57033)
                                ...                        
336636        LINESTRING (2.28519 41.683, 2.28473 41.68289)
336637      LINESTRING (2.31706 41.57213, 2.31722 41.57238)
336638    LINESTRING (2.01332 41.5798, 2.01332 41.57984,...
336639    LINESTRING (1.75909 41.24577, 1.75928 41.24584...
336640    LINESTRING (1.76369 41.24377, 1.76365 41.2438,...
Name: geometry, Length: 336641, dtype: geometry


In [77]:
def k_nearest_roads(latitud, longitud, gdf, k=5, radi=None):
    punto = Point(longitud, latitud)

    gdf = gdf.to_crs(epsg=25831)
    punto_utm = gpd.GeoSeries([punto], crs="EPSG:4326").to_crs(epsg=25831).iloc[0]

    gdf = gdf[gdf.geometry.notnull() & gdf.is_valid & ~gdf.geometry.is_empty]

    gdf['distance'] = gdf.geometry.distance(punto_utm)

    if radi is not None:
        gdf = gdf[gdf['distance'] <= radi]

    k_cercanas = gdf.nsmallest(k, 'distance')

    return k_cercanas

shapefile_path = "../Data/OpenTransportMap/roadlinks_ES511.shp"
gdf = gpd.read_file(shapefile_path)

latitud = 41.3851
longitud = 2.1734
radi = 1000

carreteras_cercanas = k_nearest_roads(latitud, longitud, gdf, k=5, radi=radi)

print(carreteras_cercanas[['roadname', 'functional', 'formofway', 'distance']])


                               roadname  functional          formofway  \
1948                Carrer de la Canuda  fifthClass            walkway   
276272  Avinguda del Portal de l'Ãngel  fifthClass            walkway   
33935   Avinguda del Portal de l'Ãngel  fifthClass            walkway   
285947              Carrer de la Canuda  fifthClass            walkway   
12060   Avinguda del Portal de l'Ãngel  fifthClass  singleCarriageway   

         distance  
1948     7.226816  
276272  17.192719  
33935   20.230763  
285947  20.230763  
12060   22.135426  


C:\Users\p2004\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [78]:
latitud = 41.3851
longitud = 2.1734

carreteras_cercanas = k_nearest_roads(latitud, longitud, gdf, k=5)

print(carreteras_cercanas[['roadname', 'functional', 'formofway', 'distance']])

C:\Users\p2004\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


                               roadname  functional          formofway  \
1948                Carrer de la Canuda  fifthClass            walkway   
276272  Avinguda del Portal de l'Ãngel  fifthClass            walkway   
33935   Avinguda del Portal de l'Ãngel  fifthClass            walkway   
285947              Carrer de la Canuda  fifthClass            walkway   
12060   Avinguda del Portal de l'Ãngel  fifthClass  singleCarriageway   

         distance  
1948     7.226816  
276272  17.192719  
33935   20.230763  
285947  20.230763  
12060   22.135426  


In [85]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Función para encontrar las 5 carreteras más cercanas
def k_nearest_roads(latitud, longitud, gdf, k=5, radi=None):
    punto = Point(longitud, latitud)

    gdf = gdf.to_crs(epsg=25831)
    punto_utm = gpd.GeoSeries([punto], crs="EPSG:4326").to_crs(epsg=25831).iloc[0]

    gdf = gdf[gdf.geometry.notnull() & gdf.is_valid & ~gdf.geometry.is_empty]
    gdf['distance'] = gdf.geometry.distance(punto_utm)

    if radi is not None:
        gdf = gdf[gdf['distance'] <= radi]

    k_cercanas = gdf.nsmallest(k, 'distance')

    return k_cercanas

# Cargar el shapefile de carreteras
shapefile_path = "../Data/OpenTransportMap/roadlinks_ES511.shp"
gdf = gpd.read_file(shapefile_path)

# Lista de ubicaciones
locations = [(41.39216, 2.009802), (41.11588, 1.191975), (41.44398, 2.237875), (41.32177, 2.082141)]

# Lista para almacenar los datos de cada ubicación
data = []

# Iterar por cada ubicación
for lat, lon in locations:
    carreteras_cercanas = k_nearest_roads(lat, lon, gdf, k=5)
    
    # Obtener valores medios para variables numéricas
    mean_values = carreteras_cercanas[['distance', 'capacity']].mean()
    
    # Seleccionar la carretera más cercana para variables categóricas
    carretera_mas_cercana = carreteras_cercanas.iloc[0]
    
    # Convertir formofway a 1 si es 'singleCarriageway' y a 0 si es 'walkway'
    formofway_value = 1 if carretera_mas_cercana['formofway'] == 'singleCarriageway' else 0
    
    # Crear un diccionario con los datos
    data_entry = {
        'latitud': lat,
        'longitud': lon,
        'formofway': formofway_value,
        'capacity': mean_values['capacity'],
        'distance': mean_values['distance'],
    }
    
    data.append(data_entry)

# Crear el DataFrame final
df_final = pd.DataFrame(data)

# Guardar en un archivo CSV
output_csv_path = "no2_concentration_dataset.csv"
df_final.to_csv(output_csv_path, index=False)

print(f"Dataset generado y guardado en: {output_csv_path}")
print(df_final)


C:\Users\p2004\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\p2004\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\p2004\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\p2004\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Dataset generado y guardado en: no2_concentration_dataset.csv
    latitud  longitud  formofway  capacity      distance
0  41.39216  2.009802          1    3276.2     12.873493
1  41.11588  1.191975          1    4799.6  35322.037789
2  41.44398  2.237875          0    3000.0     15.721151
3  41.32177  2.082141          1    3600.0     47.093358
